# Keras example

Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

In [5]:
import numpy as np

import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow.keras

2023-05-30 23:20:21.739807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 23:20:22.504937: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 23:20:22.563815: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-30 23:20:22.564009: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [6]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

exp_name = "reuters_keras"
mlflow.set_experiment(exp_name)

2023/05/30 23:20:57 INFO mlflow.tracking.fluent: Experiment with name 'reuters_keras' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', creation_time=1685469057766, experiment_id='2', last_update_time=1685469057766, lifecycle_stage='active', name='reuters_keras', tags={}>

In [7]:
mlflow.keras.autolog()

2023/05/30 23:20:59 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of keras. If you encounter errors during autologging, try upgrading / downgrading keras to a supported version, or try upgrading MLflow.
/home/madhu/miniconda3/envs/mle-dev/lib/python3.7/site-packages/mlflow/utils/autologging_utils/__init__.py:416: FutureWarning: Autologging support for keras >= 2.6.0 has been deprecated and will be removed in a future MLflow release. Use `mlflow.tensorflow.autolog()` instead.
  return _autolog(*args, **kwargs)


In [8]:
max_words = 1000
batch_size = 32
epochs = 5

mlflow.log_params({"max_words": max_words, "batch_size": batch_size, "epochs": epochs})

In [9]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
2110848/2110848 [==============================] - 0s 0us/step
8982 train sequences
2246 test sequences
46 classes


In [10]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [11]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [12]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Building model...


2023-05-30 23:21:16.632021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-30 23:21:16.632265: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-30 23:21:16.632392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TIGER02268): /proc/driver/nvidia/version does not exist
2023-05-30 23:21:16.635975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
253/253 [==============================] - 5s 14ms/step - loss: 1.4122 - accuracy: 0.6848 - val_loss: 1.0821 - val_accuracy: 0.7553
Epoch 2/5
253/253 [==============================] - 3s 12ms/step - loss: 0.7811 - accuracy: 0.8185 - val_loss: 0.9483 - val_accuracy: 0.7831
Epoch 3/5
253/253 [==============================] - 3s 12ms/step - loss: 0.5563 - accuracy: 0.8645 - val_loss: 0.8936 - val_accuracy: 0.8009
Epoch 4/5
253/253 [==============================] - 3s 11ms/step - loss: 0.4108 - accuracy: 0.8982 - val_loss: 0.8999 - val_accuracy: 0.8042
Epoch 5/5
253/253 [==============================] - 3s 11ms/step - loss: 0.3330 - accuracy: 0.9154 - val_loss: 0.8878 - val_accuracy: 0.8120


INFO:tensorflow:Assets written to: /tmp/tmpznjerfp_/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpznjerfp_/model/data/model/assets
/home/madhu/miniconda3/envs/mle-dev/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [21]:
# what is missing in the log? 

In [14]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
mlflow.log_metrics({"test_accuarcy": score[1], "test_loss": score[0]})

71/71 [==============================] - 0s 5ms/step - loss: 0.8779 - accuracy: 0.7921
Test score: 0.8778565526008606
Test accuracy: 0.7920747995376587
